# Build a Chatbot

In [1]:
import getpass
import os

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

Initiate Gemini model

In [3]:
if not os.getenv("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6b34fd7f-df14-496d-b381-e350eab75a8e-0', usage_metadata={'input_tokens': 6, 'output_tokens': 19, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}})

Model has no state though.

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="As a large language model, I have no memory of past conversations. Therefore, I don't know your name. You haven't told me!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--5cc45a9f-b7c3-4aec-8463-f2407b3c65dd-0', usage_metadata={'input_tokens': 6, 'output_tokens': 32, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})

Supposedly in this LangChain trace: https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r
 - Previous conversation not in the context
 - Model can't answer the question

So, pass the entire conversation history

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. You just told me!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--0db22638-b34c-4ed3-8809-eff5f8a36471-0', usage_metadata={'input_tokens': 22, 'output_tokens': 11, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}})

## Message Persistence

LangGraph helps keep this memory with a "persistence layer"

A a simple in-memory checkpointer helps it work with backends (PostGres, SQLite)

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Now that the `workflow` graph has been initiated with a `START` node and `model` node (with `call_model()` function), we also need to create a config to pass every time.

In [8]:
config = {"configurable": {"thread_id": "abc123"}}

Now we invoke **the application** which invokes the model

In [ ]:
# User's message/query to the chatbot
query = "Hi! I'm Bob."

# Wrap it in a HumanMessage
input_messages = [HumanMessage(query)]

# Invoke the app with the HumanMessage AND config
# Get back a conversation history as output list, latest AI message last
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! Nice to meet you. How can I help you today?


In [11]:
for mess in output["messages"]:
    mess.pretty_print() 

================================ Human Message =================================

Hi! I'm Bob.
================================== Ai Message ==================================

Hi Bob! Nice to meet you. How can I help you today?


Now when you pass a new message from the user AND the config, the model responds while knowing the entire conversation history

*Hunch: Safe to say the config "stores" the history/state...*

In [ ]:
# User's new message/query to the chatbot
query = "What's my name?"

# Wrap it in a HumanMessage
input_messages = [HumanMessage(query)]

# Invoke the app with the HumanMessage AND config
# Get back a conversation history as output list, latest AI message last
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. You told me so!


In [13]:
for mess in output["messages"]:
    mess.pretty_print() 

================================ Human Message =================================

Hi! I'm Bob.
================================== Ai Message ==================================

Hi Bob! Nice to meet you. How can I help you today?
================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob. You told me so!


In [14]:
input_messages

[HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='02db726b-6f72-466d-8a13-a490dd064af2')]

Changing the config to a different thread starts the conversation fresh

In [15]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As a large language model, I have no memory of past conversations. Therefore, I don't know your name. You haven't told me!


In [16]:
for mess in output["messages"]:
    mess.pretty_print() 

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

As a large language model, I have no memory of past conversations. Therefore, I don't know your name. You haven't told me!


We can always go back to an existing conversation thread, since its stored in a persistent database.

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

Conversation threads in the config enables different conversations with multiple users